# Particle Swarm Optimisation for Graph Neural Network Architecture Search

## step 1: load the datasets 

In [1]:
import torch
from torch_geometric.datasets import zinc
from torch_geometric.datasets import gnn_benchmark_dataset
from torch_geometric.datasets import github